<a href="http://data.bus.wisc.edu/">
        <img align="right" src="https://emaadmanzoor.com/images/color-UWcrest-print.png" height=80 style="height: 80px; float: right;"/>
</a>

# NLP+CSS 201: Controlling for Text in Causal<br/>Inference with Double Machine Learning

*By [Emaad Manzoor](http://emaadmanzoor.com), [Wisconsin School of Business](https://business.wisc.edu/)*

**The tutorial slides and video can be found [here](https://nlp-css-201-tutorials.github.io/nlp-css-201-tutorials/).**

## Install required packages

In [ ]:
!pip install doubleml==0.3.0
!pip install sklearn
!pip install statsmodels==0.10.1
!pip install econml==0.6.0

     |████████████████████████████████| 106 kB 4.0 MB/s 
     |████████████████████████████████| 8.1 MB 4.1 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
     |████████████████████████████████| 277 kB 4.1 MB/s 
     |████████████████████████████████| 110.5 MB 396 bytes/s 
     |████████████████████████████████| 6.7 MB 46.5 MB/s 
     |████████████████████████████████| 13.0 MB 90 kB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 20.1 MB 103.7 MB/s 
     |████████████████████████████████| 503 kB 69.8 MB/s 
     |████████████████████████████████| 2.9 MB 46.0 MB/s 
     |████████████████████████████████| 3.8 MB 49.5 MB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=cbd363544bd5f9bab61336fed90355857cf2c0a0df50852e218

In [ ]:
#import doubleml
import math
import numpy as np
import pandas as pd
import statsmodels.api as sm

#from econml.dml import LinearDMLCateEstimator

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF
from sklearn.exceptions import ConvergenceWarning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

from warnings import simplefilter
simplefilter("ignore", category=ConvergenceWarning)

## Fetch and relabel 20Newsgroups data, construct TF-IDF matrices

Experiment with the TfidfVectorizer parameters.

In [ ]:
newsgroup_data = fetch_20newsgroups(subset="train", remove=["headers", "footers", "quotes"], shuffle=False)

target_name_to_label = {
 'comp.graphics': 1,
 'comp.os.ms-windows.misc': 1,
 'comp.sys.ibm.pc.hardware': 1,
 'comp.sys.mac.hardware': 1,
 'comp.windows.x': 1,
 'misc.forsale': 2,
 'rec.autos': 3,
 'rec.motorcycles': 3,
 'rec.sport.baseball': 4,
 'rec.sport.hockey': 4,
 'sci.crypt': 5,
 'sci.electronics': 6,
 'sci.med': 7,
 'sci.space': 8,
 'alt.atheism': 9,
 'soc.religion.christian': 9,
 'talk.religion.misc': 9, 
 'talk.politics.guns': 10,
 'talk.politics.mideast': 10,
 'talk.politics.misc': 10,
}
label_names = ["comp", "sale", "auto", "sport", "crypt", "electronics", "med", "space", "religion", "politics"]
labels = [target_name_to_label[newsgroup_data.target_names[original_label]]
              for original_label in newsgroup_data.target]
labels = np.array(labels)

vectorizer = TfidfVectorizer(min_df=25, max_df=0.01)
tfidf_vectors = vectorizer.fit_transform(newsgroup_data.data).toarray()
print(tfidf_vectors.shape, labels.shape)

documents = []
dropped_document_idx = set([])
vocabset = set(vectorizer.get_feature_names())
for idx, text in enumerate(newsgroup_data.data):
    document = []
    for token in text.split(" "):
        if token not in vocabset:
            continue
        document.append(token)

    if len(document) == 0:
        dropped_document_idx.add(idx)
        continue
    
    document = " ".join(document)
    if len(document) > 1000:
        document = document[:1000]

    documents.append(document)

tfidf_vectors = np.array([tfidf_vectors[idx, :]
                          for idx in range(tfidf_vectors.shape[0])
                          if idx not in dropped_document_idx])
labels = np.array([labels[idx]
                   for idx in range(len(labels))
                   if idx not in dropped_document_idx])
print(tfidf_vectors.shape, labels.shape, len(documents))

(11314, 3882) (11314,)
(9816, 3882) (9816,) 9816


## Simulate Data for Treatment Effects

For each document $i$, a row of the data consists of:

   * Unobserved confounder $U_i$: Binary, equals 1 if text is from a newsgroup on religion
   * Treatment $Z_i$: depends on $U_i$
   * Outcome $Y_i$: Real-valued, depends on $Z_i$ and $U_i$
   * Document TF-IDF vector $\pmb{X}_i$
   
**The true treatment effect is 0.05.**

You can play around with the manner of simulation (eg. increase the treatment effect size).

In [ ]:
def sigmoid(x):
    return 1. / (1. + math.exp(-x))

simulated_data = []
for idx in tqdm(range(tfidf_vectors.shape[0])):    
    # confounder
    U = int(labels[idx]==9) # unobserved confounder = religion topic, influences text
        
    # treatment
    Z = 2.*U + np.random.normal(loc=0.5, scale=5.0)  # Z depends on U
    
    # binary outcome
    # yprob = sigmoid(-0.5 + 0.05*Z + 5.0*U)
    # Y = np.random.choice(a=[0, 1],  p=[1.0 - yprob, yprob])
    Y = -0.5 + 0.05*Z + 5.0*U + np.random.normal(0.0, 0.1)
    
    simulated_data.append([Y, Z, U] + list(tfidf_vectors[idx]))

simulated_data = np.array(simulated_data)

  0%|          | 0/9816 [00:00<?, ?it/s]

## Convert to Pandas Dataframe, Describe Data

In [ ]:
df = pd.DataFrame(simulated_data)
df.columns = ["Y", "Z", "U"] + ["Word" + str(i) for i in range(tfidf_vectors.shape[1])]
df

,Y,Z,U,Word0,Word1,Word2,Word3,Word4,Word5,Word6,Word7,Word8,Word9,Word10,Word11,Word12,Word13,Word14,Word15,Word16,Word17,Word18,Word19,Word20,Word21,Word22,Word23,Word24,Word25,Word26,Word27,Word28,Word29,Word30,Word31,Word32,Word33,Word34,Word35,Word36,...,Word3842,Word3843,Word3844,Word3845,Word3846,Word3847,Word3848,Word3849,Word3850,Word3851,Word3852,Word3853,Word3854,Word3855,Word3856,Word3857,Word3858,Word3859,Word3860,Word3861,Word3862,Word3863,Word3864,Word3865,Word3866,Word3867,Word3868,Word3869,Word3870,Word3871,Word3872,Word3873,Word3874,Word3875,Word3876,Word3877,Word3878,Word3879,Word3880,Word3881
0,0.183011,11.756561,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,-0.085405,8.609010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.148449,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.102486,10.239682,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024428,0.025255,0.0,0.0,0.0,0.050249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021736,0.0,0.0,0.0,0.0,0.0,0.025255,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024428,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.021589,0.0
3,-0.066852,9.093971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,4.539129,1.899150,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9811,-0.434235,-0.025587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9812,-0.254999,8.813957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9813,-0.780463,-4.592288,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9814,-0.341115,0.988236,0.

In [ ]:
df[["Y", "Z", "U"]].describe()

,Y,Z,U
count,9816.000000,9816.000000,9816.000000
mean,0.198048,0.702752,0.132641
std,1.750072,5.038643,0.339203
min,-1.490108,-17.178154,0.000000
25%,-0.633092,-2.739312,0.000000
50%,-0.426790,0.663566,0.000000
75%,-0.175845,4.163009,0.000000
max,5.439211,20.845121,1.000000


## Treatment Effect Estimation 1: Regress $Y_i$ on $Z_i$ and $U_i$

Since the estimation includes the unobserved confounder $U_i$, the treatment effect can
be estimated without bias (there are no unobserved confounders).

In [ ]:
%%time
y = simulated_data[:, 0]
X = sm.add_constant(simulated_data[:, 1:3])
model = sm.OLS(endog=y, exog=X)
res = model.fit(method="pinv", maxiter=2000)
print(res.summary(yname="Y", xname=["const", "Z", "U"]))

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 1.491e+06
Date:                Thu, 11 Nov 2021   Prob (F-statistic):               0.00
Time:                        15:22:10   Log-Likelihood:                 8662.7
No. Observations:                9816   AIC:                        -1.732e+04
Df Residuals:                    9813   BIC:                        -1.730e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5011      0.001   -460.033      0.0

## Treatment Effect Estimation 2: Regress $Y_i$ on $Z_i$ only

Since the estimation does not include the unobserved confounder $U_i$, the estimated treatment effect will
be biased.

In [ ]:
%%time
y = simulated_data[:, 0]
X = sm.add_constant(simulated_data[:, 1:2])
model = sm.OLS(endog=y, exog=X)
res = model.fit(method="pinv", maxiter=100)
print(res.summary(yname="Y", xname=["const", "Z"]))

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     763.9
Date:                Thu, 11 Nov 2021   Prob (F-statistic):          5.45e-162
Time:                        15:22:20   Log-Likelihood:                -19044.
No. Observations:                9816   AIC:                         3.809e+04
Df Residuals:                    9814   BIC:                         3.811e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1305      0.017      7.600      0.0

## Treatment Effect Estimation 3: Regress $Y_i$ on $Z_i$ and $\pmb{X}_i$

Do not run this, it takes too long, and may not even converge!

In [ ]:
# %%time
# y = simulated_data[:, 0]
# X = sm.add_constant(np.hstack((simulated_data[:, 1:2], simulated_data[:, 3:])))
# model = sm.OLS(endog=y, exog=X)
# res = model.fit(method="pinv", maxiter=200)
# res.summary(yname="Y", xname=["const", "Z"])

## Treatment Effect Estimation 4: Regress $Y_i$ on $Z_i$ and Document-Topic Weights

Play around with the number of topics.

In [ ]:
%%time
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

numtopics = 50
nmf = NMF(n_components=numtopics).fit(tfidf_vectors)
tfidf_feature_names = vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, 10)
nmf_topic_weights = nmf.transform(tfidf_vectors)

Topic #0: geb cadre dsl chastity n3jxp pitt intellect surrender shameful skepticism
Topic #1: 3d object animation ray surface replies objects energy polygon libraries
Topic #2: armenian armenians turkish armenia genocide russian soviet azerbaijan turkey turks
Topic #3: motherboard cpu mhz fpu upgrade cache 386 slot processor nubus
Topic #4: motif x11r5 xt toolkit interviews linux icon openwindows pixmap fixing
Topic #5: pittsburgh detroit rangers leafs montreal islanders espn playoffs cup devils
Topic #6: sin heaven spirit scripture mary eternal holy catholic matthew kingdom
Topic #7: ide hd isa transfer bios sec boot mfm burst slave
Topic #8: msg chinese foods eat reaction taste brain effects salt sick
Topic #9: vga svga monitors lc modes compatible recommend adapter connect thanx
Topic #10: moon launch orbit lunar shuttle mission satellite spacecraft solar mars
Topic #11: objective morality subjective absolute animals goals relative observations immoral species
Topic #12: motorcycle 

In [ ]:
%%time
y = simulated_data[:, 0]
X = sm.add_constant(np.hstack((simulated_data[:, 1:2], nmf_topic_weights)))
model = sm.OLS(endog=y, exog=X)
res = model.fit(method="pinv", maxiter=200)
print(res.summary(xname=["const", "Z"] + ["Topic" + str(i+1) for i in range(numtopics)]))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.533
Model:                            OLS   Adj. R-squared:                  0.531
Method:                 Least Squares   F-statistic:                     218.6
Date:                Thu, 11 Nov 2021   Prob (F-statistic):               0.00
Time:                        15:24:27   Log-Likelihood:                -15674.
No. Observations:                9816   AIC:                         3.145e+04
Df Residuals:                    9764   BIC:                         3.183e+04
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0730      0.029      2.512      0.0

## Treatment Effect Estimation 5: Regress $Y_i$ on $Z_i$ and $\pmb{X}_i$ using Double Machine Learning

Play around with the type of ML models used to predict the treatment and outcome.

Note that some models take really long to train (eg. Random Forests).

### Using the [EconML](https://econml.azurewebsites.net/) package

In [ ]:
%%time
Y = simulated_data[:, 0].ravel() # outcome
T = simulated_data[:, 1] # treatment
W = simulated_data[:, 3:] # text

dml_mult = LinearDMLCateEstimator(model_y=LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1),
                                  model_t=LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1),
                                  linear_first_stages=True, n_splits=2)

dml_mult.fit(Y=Y, T=T, W=W, inference="statsmodels")
te_pred_mult = dml_mult.const_marginal_effect().ravel()

lb_mult, ub_mult = dml_mult.const_marginal_effect_interval(alpha=0.05)
lb_mult, ub_mult = lb_mult.ravel(), ub_mult.ravel()

print("Treatment Effect, 95% CI" + "\t\t" + "{:.6f}".format(te_pred_mult[0]) +\
      "({:.6f}".format(lb_mult[0]) + ", " + "{:.6f})".format(ub_mult[0]))

Treatment Effect, 95% CI		0.050828(0.047201, 0.054456)
CPU times: user 3min 52s, sys: 10.7 s, total: 4min 3s
Wall time: 2min 39s


### Using the [DoubleML](https://docs.doubleml.org/stable/index.html) package

In [ ]:
%%time
data = doubleml.DoubleMLData(data=df, y_col="Y", d_cols="Z",
                             x_cols=["Word" + str(i) for i in range(len(df.columns)-3)])

treatment_predictor = LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1)
outcome_predictor = LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1)

estimator = doubleml.DoubleMLPLR(data, treatment_predictor, outcome_predictor,
                                 apply_cross_fitting=False, n_folds=2)
estimator.fit()
print(estimator.summary)

       coef   std err          t         P>|t|     2.5 %    97.5 %
Z  0.040553  0.002515  16.123596  1.741738e-58  0.035623  0.045482
CPU times: user 1min 6s, sys: 1.36 s, total: 1min 8s
Wall time: 1min 49s


### From Scratch

Note: Standard errors are different from those recommended in the double machine learning paper.

In [ ]:
%%time
simulated_data_train, simulated_data_inference = train_test_split(simulated_data, test_size=0.5)

Y_train = simulated_data_train[:, 0]
T_train = simulated_data_train[:, 1]
text_train = simulated_data_train[:, 3:]

Y_inference = simulated_data_inference[:, 0]
T_inference = simulated_data_inference[:, 1]
text_inference = simulated_data_inference[:, 3:]

CPU times: user 104 ms, sys: 1.87 ms, total: 106 ms
Wall time: 111 ms


In [ ]:
%%time
model_y_tr = LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1).fit(X=text_train, y=Y_train)
model_t_tr = LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1).fit(X=text_train, y=T_train)

CPU times: user 2min 13s, sys: 10.3 s, total: 2min 24s
Wall time: 1min 32s


In [ ]:
%%time
model_y_inf = LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1).fit(X=text_inference, y=Y_inference)
model_t_inf = LassoCV(cv=2, n_alphas=1, verbose=0, n_jobs=-1).fit(X=text_inference, y=T_inference)

CPU times: user 2min 7s, sys: 7.71 s, total: 2min 15s
Wall time: 1min 25s


In [ ]:
yres1 = Y_inference - model_y_tr.predict(text_inference)
tres1 = T_inference - model_t_tr.predict(text_inference)
yres2 = Y_train - model_y_inf.predict(text_train)
tres2 = T_train - model_t_inf.predict(text_train)

In [ ]:
theta_1 = np.mean(yres1*tres1)/np.mean(tres1**2)
theta_2 = np.mean(yres2*tres2)/np.mean(tres2**2)
theta = 0.5 * (theta_1 + theta_2)
print("Cross-Fitted Treatment Effect", theta)

Cross-Fitted Treatment Effect 0.05048492653618829


In [ ]:
print("Sample-Split Treatment Effect")
regression_model = sm.OLS(endog=yres1, exog=tres1, hasconst=False)
res = regression_model.fit(method="pinv", maxiter=100)
print(res.summary())

Sample-Split Treatment Effect
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.069
Model:                            OLS   Adj. R-squared (uncentered):              0.069
Method:                 Least Squares   F-statistic:                              363.8
Date:                Thu, 11 Nov 2021   Prob (F-statistic):                    2.67e-78
Time:                        15:33:04   Log-Likelihood:                         -10071.
No. Observations:                4908   AIC:                                  2.014e+04
Df Residuals:                    4907   BIC:                                  2.015e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------